In [ ]:
def wrapper():
    import os
    import random
    from hops import model
    model_path = os.getcwd() + '/model'
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    model.export(model_path, "mnist", metrics={'acc': random.randrange(10000)})
    model.export(model_path, "mnist", metrics={'acc': random.randrange(10000)})
    model.export(model_path, "mnist", metrics={'acc': random.randrange(10000)})
    model.export(model_path, "cifar", metrics={'acc': random.randrange(10000)})

In [ ]:
from hops import experiment
experiment.launch(wrapper)